In [3]:
# !pip install ray
# !pip install "ray[default]"
# !pip uninstall grpcio
#!pip install lxml
import requests
import json
import random
from time import time, sleep
from bs4 import BeautifulSoup
from tqdm import tqdm
import ray
basic_url = "https://www.10000recipe.com"
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
ingridient_seperator = "                                                                                    "

In [4]:
# ray for loop 안에서 잦은 get 지양

In [5]:
# lxml 사용시 htmlParseEntityRef: expecting ';' warning 뜸
# 실행은 되지만 warning 문을 출력해서 log에 뜸
# 그래서 html.parser 사용

In [6]:
@ray.remote
def crawling_recipe_path_list(page):
    recipe_path_list = []
    url = f"{basic_url}/recipe/list.html?order=date&page={page}"
    response = requests.get(url, headers = headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        print(f"{page} 페이지 접속 불가")
        return []
    soup = BeautifulSoup(response.text, "html.parser")
    try:
        for a in soup.find("ul","common_sp_list_ul").find_all("a","common_sp_link"):
            recipe_path_list.append(a.get("href"))
    except:
        print(f"{page}페이지 에러 뜸 / 빈 리스트 반환")
        return []
        
    return recipe_path_list

In [7]:
# try catch로 오류 처리 해주기
def parsing_recipe_ingredients(soup):
    ingredient_list = []
    for ul in (soup.find("div", "ready_ingre3").find_all("ul")):
        for li in ul.find_all("li"):
            ingredient = {}
            ingredient['name'] = (li.text.split("\n")[0].split(ingridient_seperator)[0].strip(" "))
            ingredient['quantity'] = (li.find("span","ingre_unit").text.strip(" "))
            ingredient_list.append(ingredient)
    return ingredient_list

In [8]:
# try catch로 오류 처리 해주기
def parsing_recipe_ingredients_new(soup):
    ingredient_list = []
    for dl in (soup.find("div", "cont_ingre").find_all("dl")):
        for ingre in dl.find("dd").text.split(", "):
            name , quantity = ingre.split()
            ingredient = {}
            ingredient['name'] = name
            ingredient['quantity'] = quantity
            ingredient_list.append(ingredient)
    return ingredient_list
# response = requests.get(f"https://www.10000recipe.com{recipe_path_list[6]}", headers=headers)
# print(recipe_path_list[2])
# soup = BeautifulSoup(response.text, "html.parser")
# parsing_recipe_ingredients_new(soup)

In [9]:
def parsing_recipe_sequences(soup):
    sequence_list = []
    for idx, div in enumerate(soup.find("div","view_step").find_all("div","view_step_cont")):
        sequence = {}
        sequence['sequence'] = idx+1
        sequence['content'] = div.text
        sequence_list.append(sequence)
    return sequence_list

In [10]:
@ray.remote
def makeing_recipe(recipe_path):
    sleep(random.randrange(1,2)*random.random())
    url = basic_url+recipe_path
    response = requests.get(url, headers=headers)
    # dag만들 때 오류처리 하기
    if response.status_code !=200:
        print(f"페이지 오류 {recipe_path}")
        return {}
    soup = BeautifulSoup(response.text, "html.parser")
    
    recipe = {}
    try:
        recipe['recipe_id'] = recipe_path.split("/")[2]
    except:
        print(f"아이디 오류 {recipe_path}")
    try: 
        recipe['img'] = soup.find("img", id="main_thumbs").get("src")
    except:
        print(f"이미지 오류 {recipe_path}")
    try:
        recipe['title'] = soup.find("div","view2_summary st3").find("h3").text
    except:
        print(f"이름 오류 {recipe_path}")
        
    try:
        recipe['size'] = soup.find("span","view2_summary_info1").text
    except:
        print(f"몇인분 오류 {recipe_path}")
        
    try:
        recipe['ingredients'] = parsing_recipe_ingredients(soup)
    except:
        try:
            recipe['ingredients'] = parsing_recipe_ingredients_new(soup)
        except:
            return {} # 그냥 레시피 버리기
            print(f"재료 오류 {recipe_path}")
        
    try:
        recipe['sequences'] = parsing_recipe_sequences(soup)
    except:
        print(f"조리법 오류 {recipe_path}")
    
    return recipe

In [11]:
ray.init()

2022-08-05 11:38:54,382	INFO services.py:1456 -- View the Ray dashboard at http://127.0.0.1:8265


RayContext(dashboard_url='127.0.0.1:8265', python_version='3.9.13', ray_version='1.12.1', ray_commit='4863e33856b54ccf8add5cbe75e41558850a1b75', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-05_11-38-52_673324_24237/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-05_11-38-52_673324_24237/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-08-05_11-38-52_673324_24237', 'metrics_export_port': 51814, 'gcs_address': '127.0.0.1:53303', 'address': '127.0.0.1:53303', 'node_id': '0386b7e8eaf5b2f375652cd48913c294e6362af58b1b88e100c73627'})

In [12]:
def crawling(st,end):
    recipe_list = []
    for i in tqdm(range(st,end)):
        recipe_path_list = ray.get(crawling_recipe_path_list.remote(i))
        start = time()
        ref_list = []
        for recipe_path in tqdm(recipe_path_list):
            ref_list.append(makeing_recipe.remote(recipe_path))
        recipe_list.extend(ray.get(ref_list))
        c = (time()-start)
        print(c)
    with open(f"recipe/Asite/{st}_{end}.json", 'w',encoding = 'UTF-8') as f:
        json.dump(recipe_list, f,ensure_ascii = False)
    return recipe_list

In [14]:
ray.shutdown()